<a href="https://colab.research.google.com/github/opeyemiferanmi1/Hamoye-Task1-Understanding-Electricity-Utility-data-/blob/master/Stage_B_Predicting_Energy_Efficiency_of_Buildings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
cols_to_drop = data[['date','lights']]#columns to drop
df= data.drop(cols_to_drop, axis=1)
df.head(2)

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,60,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,45.566667,17.166667,55.2,7.026667,84.256667,17.2,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,45.992500,17.166667,55.2,6.833333,84.063333,17.2,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [4]:
# Normalize the data set to a common scale using min max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
norm_df = pd.DataFrame(scaler.fit_transform(df),columns= df.columns)
norm_df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,0.381691,0.381070,0.841827,0.170594,0.653428,0.173329,0.661412,0.223032,0.677290,0.372990,0.097674,0.894737,0.500000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,0.381691,0.375443,0.839872,0.170594,0.651064,0.173329,0.660155,0.226500,0.678532,0.369239,0.100000,0.894737,0.476190,0.894872,0.533937,0.372083,0.372083
2,0.037383,0.32735,0.530502,0.225345,0.679445,0.215188,0.755569,0.344745,0.778062,0.175506,0.380037,0.367487,0.830704,0.170594,0.646572,0.173329,0.655586,0.219563,0.676049,0.365488,0.102326,0.894737,0.452381,0.835897,0.529412,0.572848,0.572848
3,0.037383,0.32735,0.524080,0.225345,0.678414,0.215188,0.758685,0.341441,0.770949,0.175506,0.380037,0.363800,0.833401,0.164310,0.641489,0.164175,0.650788,0.219563,0.671909,0.361736,0.104651,0.894737,0.428571,0.776923,0.524887,0.908261,0.908261
4,0.046729,0.32735,0.531419,0.225345,0.676727,0.215188,0.758685,0.341441,0.762697,0.178691,0.380037,0.361859,0.848264,0.170594,0.639362,0.164175,0.650788,0.219563,0.671909,0.357985,0.106977,0.894737,0.404762,0.717949,0.520362,0.201611,0.201611


## Simple Linear Regression

In [7]:
#select a sample of the dataset
import numpy as np
x = np.array(df['T2']).reshape(-1, 1)
y = df['T6']

In [8]:
#Split data into test and train data set
from sklearn.model_selection import train_test_split
sx_train, sx_test, sy_train, sy_test = train_test_split(x,y, test_size =0.3, random_state =42)

In [9]:
from sklearn.linear_model import LinearRegression
simple_linear_model = LinearRegression()
#fit
simple_linear_model.fit(sx_train, sy_train)
#obtain predictions
s_predicted_values = simple_linear_model.predict(sx_test)

In [10]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
def metrics(true,predicted_values):
    mae = print('MAE:{:.2f}'.format(mean_absolute_error(true,predicted_values)))
    r_squared = print('R_squared:{:.2f}'.format(r2_score(true,predicted_values)))
    rss = print('RSS:{:.2f}'.format(np.sum(np.square(true-predicted_values))))
    rmse = print('RMSE:{:.3f}'.format(np.sqrt(mean_squared_error(true,predicted_values))))

In [11]:
metrics(sy_test,s_predicted_values)

MAE:2.82
R_squared:0.64
RSS:78033.97
RMSE:3.630


## Multiple Regression

In [12]:
features = norm_df.drop(['Appliances'],axis =1)
target = norm_df['Appliances']

In [13]:
#Split data into test and train data set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features,target, test_size =0.3, random_state =42)

### Linear Regression

In [14]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
#fit
linear_model.fit(x_train, y_train)
#obtain predictions
predicted_values = linear_model.predict(x_test)

In [15]:
metrics(y_test,predicted_values)

MAE:0.05
R_squared:0.15
RSS:45.35
RMSE:0.088


In [16]:
def get_weights_df(model, feat, col_name):
    '''This function returns the weight of every feature'''
    weights = pd.Series(model.coef_,feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns =['Features',col_name]
    weights_df[col_name].round(3)
    return weights_df

In [44]:
linear_model_weights = get_weights_df(linear_model,x_train,'Linear_model_Weights').sort_values('Linear_model_Weights')
linear_model_weights

,Features,Linear_model_Weights
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


### Ridge Regression

In [39]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
#obtain predictions
r_predicted_values = ridge_reg.predict(x_test)

In [40]:
metrics(y_test,r_predicted_values)

MAE:0.05
R_squared:0.15
RSS:45.37
RMSE:0.088


### Lasso Regression

In [32]:
import numpy as np
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(x_train, y_train)
#obtain predictions
l_predicted_values = lasso_reg.predict(x_test)

In [33]:
metrics(y_test,l_predicted_values)

MAE:0.06
R_squared:0.03
RSS:51.85
RMSE:0.094


In [36]:
lasso_weights_df = get_weights_df(lasso_reg,x_train,'Lasso_Weights')
lasso_weights_df

,Features,Lasso_Weights
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [41]:
#count non_zeros feature weights
lasso_weights_df[lasso_weights_df['Lasso_Weights']!=0]

,Features,Lasso_Weights
0,RH_out,-0.049557
1,RH_8,-0.000110
24,Windspeed,0.002912
25,RH_1,0.017880
